In [1]:
pip install transformers sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [3]:
conda install -c conda-forge sentencepiece


Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - sentencepiece


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-3.4.0              |       h39f12f2_0         2.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.8 MB

The following packages will be UPDATED:

  openssl                                  3.3.2-h8359307_0 --> 3.4.0-h39f12f2_0 



                                                                                
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [36]:
from transformers import MarianMTModel, MarianTokenizer
from sentence_transformers import SentenceTransformer, util

# Load the paraphrase model
paraphrase_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Load the translation models and tokenizers (English to Hindi and English to French)
translation_model_hi = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-hi')
translation_tokenizer_hi = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-hi')

translation_model_fr = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
translation_tokenizer_fr = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')

def translate_text(text, target_lang):
    """Translates text from English to the specified target language (either 'hi' for Hindi or 'fr' for French)."""
    # Set the appropriate model and tokenizer based on target language
    if target_lang == 'hi':
        model, tokenizer = translation_model_hi, translation_tokenizer_hi
    elif target_lang == 'fr':
        model, tokenizer = translation_model_fr, translation_tokenizer_fr
    else:
        raise ValueError("Unsupported language code. Use 'hi' for Hindi or 'fr' for French.")

    # Translate the text
    text = f">>{target_lang}<< {text}"
    tokenized_text = tokenizer(text, return_tensors="pt", padding=True)
    translated = model.generate(**tokenized_text)
    return tokenizer.decode(translated[0], skip_special_tokens=True).replace(" ?", "?").replace(" !", "!")

def check_paraphrase(text1, text2):
    """Checks if two texts are paraphrases using a sentence similarity model."""
    embeddings1 = paraphrase_model.encode(text1, convert_to_tensor=True)
    embeddings2 = paraphrase_model.encode(text2, convert_to_tensor=True)
    similarity_score = util.pytorch_cos_sim(embeddings1, embeddings2).item()
    return similarity_score > 0.8  # Assuming a threshold of 0.8 for paraphrase similarity

# Example usage
text1 = "my name is  kshitij"
text2 = "Hi, how are you?"

# Step 1: Check if texts are paraphrases
is_paraphrase = check_paraphrase(text1, text2)
print(f"Are the texts paraphrases? {'Yes' if is_paraphrase else 'No'}")

# Step 2: Translate text1 into Hindi and French
translated_text_hi = translate_text(text1, 'hi')
translated_text_fr = translate_text(text1, 'fr')
print(f"Translated Text in Hindi: {translated_text_hi}")
print(f"Translated Text in French: {translated_text_fr}")


Are the texts paraphrases? No
Translated Text in Hindi: जिम्बाबवे मेरा नाम है खशीज
Translated Text in French: Mon nom est Kshitij
